<a href="https://colab.research.google.com/github/kieunseo/ML/blob/main/Regression/Wine_%ED%80%84%EB%A6%AC%ED%8B%B0%EC%98%88%EC%B8%A1_%EC%98%B5%ED%8A%9C%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/wine.csv")
display(df)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  class                 6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [ ]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
class                   0
dtype: int64

In [ ]:
data = df[['volatile acidity', 'chlorides', 'density', 'alcohol', 'quality']]

In [ ]:
data.corr()

,volatile acidity,chlorides,density,alcohol,quality
volatile acidity,1.000000,0.377124,0.271296,-0.037640,-0.265699
chlorides,0.377124,1.000000,0.362615,-0.256916,-0.200666
density,0.271296,0.362615,1.000000,-0.686745,-0.305858
alcohol,-0.037640,-0.256916,-0.686745,1.000000,0.444319
quality,-0.265699,-0.200666,-0.305858,0.444319,1.000000


In [ ]:
data.isnull().sum()

volatile acidity    0
chlorides           0
density             0
alcohol             0
quality             0
dtype: int64

In [ ]:
#데이터 스플릿
from sklearn.model_selection import train_test_split
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_vaild, y_train, y_vaild = train_test_split(X, y, test_size=0.2)


#스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_vaild_scaled = scaler.transform(X_vaild)

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
# 모델학습 (with 옵튜나)
# 옵튜나 -> 목적함수

import optuna
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


# 목적함수
# 의사결정회귀 하이퍼 파라미터들 값 지정
def objective(trial):
    param = {
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
    }

    #학습 모델 생성
    model = DecisionTreeRegressor(**param)
    model.fit(X_train_scaled, y_train) #학습 진행

    pred = model.predict(X_vaild_scaled)
    score = mean_squared_error(pred, y_vaild)
    return score**(1/2)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
best_params = study.best_params

[I 2024-07-01 04:53:53,069] A new study created in memory with name: no-name-e1885e7e-9668-4ef8-be19-4773cafada35
[I 2024-07-01 04:53:53,085] Trial 0 finished with value: 0.794460558599356 and parameters: {'min_samples_leaf': 6, 'max_depth': 1}. Best is trial 0 with value: 0.794460558599356.
[I 2024-07-01 04:53:53,093] Trial 1 finished with value: 0.794460558599356 and parameters: {'min_samples_leaf': 7, 'max_depth': 1}. Best is trial 0 with value: 0.794460558599356.
[I 2024-07-01 04:53:53,100] Trial 2 finished with value: 0.794460558599356 and parameters: {'min_samples_leaf': 4, 'max_depth': 1}. Best is trial 0 with value: 0.794460558599356.
[I 2024-07-01 04:53:53,109] Trial 3 finished with value: 0.794460558599356 and parameters: {'min_samples_leaf': 3, 'max_depth': 1}. Best is trial 0 with value: 0.794460558599356.
[I 2024-07-01 04:53:53,129] Trial 4 finished with value: 0.7991763465518915 and parameters: {'min_samples_leaf': 2, 'max_depth': 10}. Best is trial 0 with value: 0.794460

In [ ]:
model = DecisionTreeRegressor(**study.best_params)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_vaild_scaled)

print('MSE: ', mean_squared_error(y_vaild, y_pred))
print('RMSE: ', mean_squared_error(y_vaild, y_pred)**(1/2))
model.feature_importances_

MSE:  0.5383528788865893
RMSE:  0.7337253429496553


array([0.25847304, 0.08424487, 0.07106241, 0.58621968])